In [1]:
import json, sys, pathlib
from typing import List, Dict

In [4]:
# -----------------------------------------------------------------
COURSE_ROOT = pathlib.Path("../API_course_data_filtered")   # 总课程目录
INPUT_FILE  = pathlib.Path("input.json")                 # 前端写入
OUTPUT_FILE = pathlib.Path("output.json")                # 后端输出


In [7]:
# 2. 读取前端输入
# -----------------------------------------------------------------
params = json.loads(INPUT_FILE.read_text(encoding="utf-8"))
PROGRAM = params["program"].upper()           # 如 'SYDE', 'ECE'
SPEC    = params["specialization"]
START   = params["start_term"]
N_TERM  = params["num_terms"]


In [9]:
# 3. 读取专业规则文件 ./<program>.json
# -----------------------------------------------------------------
RULES_PATH = pathlib.Path(f"{PROGRAM.lower()}.json")
if not RULES_PATH.exists():
    sys.exit(f"[ERROR] Rule file {RULES_PATH} not found.")

rules_all = json.loads(RULES_PATH.read_text(encoding="utf-8"))
if SPEC not in rules_all:
    sys.exit(f"[ERROR] Unknown specialization {SPEC} in {RULES_PATH}.")
cfg = rules_all[SPEC]

In [10]:
# 4. 学期序列工具
# -----------------------------------------------------------------
TERM_MAP = {1: "Winter", 5: "Spring", 9: "Fall"}

def next_term_code(code: str) -> str:
    """'1251' → '1255' → '1259' → '1261' ..."""
    yy, d = int(code[1:3]), int(code[3])
    nd = {1:5, 5:9, 9:1}[d]
    if nd == 1: yy += 1
    return f"1{yy:02d}{nd}"

def term_sequence(start: str, n: int) -> List[str]:
    seq = [start]
    while len(seq) < n:
        seq.append(next_term_code(seq[-1]))
    return seq

TERMS = term_sequence(START, N_TERM)


In [12]:
# 5. 加载课程开课信息（仅在对应院系文件夹搜索）
# -----------------------------------------------------------------
def load_offerings(program: str, term_codes: List[str]) -> Dict[str, List[str]]:
    base = COURSE_ROOT / program
    if not base.is_dir():
        sys.exit(f"[ERROR] {base} not found.")
    offers: Dict[str, List[str]] = {}
    for tc in term_codes:
        file_path = base / f"{tc}.json"
        if not file_path.exists():
            continue
        for course in json.loads(file_path.read_text()):
            code = f"{course['subjectCode']} {course['catalogNumber']}"
            offers.setdefault(code, []).append(tc)
    return offers

offers = load_offerings(PROGRAM, TERMS)

In [13]:
# 6. 处理规则：固定必修、OR 组、Elective
# ---------------------------------]-------------------------------
def choose_from_or(or_groups):
    picks = []
    for group in or_groups:
        # 优先挑选当前时间窗内可修课程，否则默认取第 1 门
        avail = [c for c in group if c in offers]
        picks.append(avail[0] if avail else group[0])
    return picks

required = cfg["specified_fixed"] + choose_from_or(cfg.get("specified_or", []))
elective_pool = cfg["elective"]["list"]
elective_min  = cfg["elective"]["min_count"]

In [14]:
def wrap(code):
    return {
        "code": code,
        "offered_terms": offers.get(code, [])
    }

result = {
    "terms": TERMS,
    "required":  [wrap(c) for c in required],
    "elective":  [wrap(c) for c in elective_pool],
    "elective_min": elective_min,
    "other": []                       # 预留
}


In [16]:
# 7. 写出 output.json
# -----------------------------------------------------------------
OUTPUT_FILE.write_text(json.dumps(result, indent=2), encoding="utf-8")
print(f"[OK] Course pool written to {OUTPUT_FILE}")
print(OUTPUT_FILE)

[OK] Course pool written to output.json
output.json
